In [1]:
import cv2
import pygame
import numpy as np
from scipy.spatial import distance as dist
import mediapipe as mp

# Initialize Pygame mixer for alarm sound
pygame.mixer.init()
sound = pygame.mixer.Sound('beep-warning-6387.mp3')

# Constants
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 100  # 10 seconds at 30 fps
TRANSPARENCY = 0.4  # Transparency factor for the overlay

# Function to compute the eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

# Initialize video capture
cap = cv2.VideoCapture(0)
frame_counter = 0
drowsy = False

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            leftEye = [face_landmarks.landmark[i] for i in [33, 160, 158, 133, 153, 144]]
            rightEye = [face_landmarks.landmark[i] for i in [362, 385, 387, 263, 373, 380]]
            
            leftEye = [(int(p.x * frame.shape[1]), int(p.y * frame.shape[0])) for p in leftEye]
            rightEye = [(int(p.x * frame.shape[1]), int(p.y * frame.shape[0])) for p in rightEye]
            
            # Remove the polylines drawing code
            
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            
            ear = (leftEAR + rightEAR) / 2.0
            
            if ear < EYE_AR_THRESH:
                frame_counter += 1
                if frame_counter >= EYE_AR_CONSEC_FRAMES:
                    if not drowsy:
                        sound.play()
                        drowsy = True
                    # Create a transparent overlay
                    overlay = frame.copy()
                    overlay[:] = (0, 0, 255)  # Red overlay
                    frame = cv2.addWeighted(overlay, TRANSPARENCY, frame, 1 - TRANSPARENCY, 0)
                    cv2.putText(frame, "DROWSINESS DETECTED", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            else:
                frame_counter = 0
                if drowsy:
                    sound.stop()
                    drowsy = False
                # Add active status indicator
                cv2.putText(frame, "ACTIVE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        # If no face is detected, show "No Face Detected" message
        cv2.putText(frame, "No Face Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow('Drowsiness Detection', frame)
    
    # Check for 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\mukul\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
